<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize-extract-genQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO : OPTIMIZEEEEEE

# https://turbolab.in/abstractive-summarization-using-googles-t5/

#OPTIMIZE
# https://www.youtube.com/watch?v=2glb3BfOSVE

In [1]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git

     |████████████████████████████████| 3.5 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 6.8 MB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.5 MB/s 
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-t1w4tae8
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-t1w4tae8
     |████████████████████████████████| 235 kB 4.3 MB/s 
  Created wheel for pke: filename=pke-1.8.1-py3-none-any.whl size=8764192 sha256=f0b02183e654bb7033d59898f0efa26c2b91c8aa9113b489db56275c92efe27d
  Stored in directory: /tmp/pip-ephem-wheel-cache-m7hfsuwf/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
Succes

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pke
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
T5_MODEL_PATH = 't5-large'

model = T5ForConditionalGeneration.from_pretrained(T5_MODEL_PATH)
tokenizer = T5Tokenizer.from_pretrained(T5_MODEL_PATH)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:01<?, ?B/s]

In [33]:
def summarize(text):
  # preprocess
  text = text.strip().replace("\n","")
  text = text.replace("-", " ")
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  input_tokens = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length= 1000, padding='max_length', truncation=True)

  summary_encoded = model.generate(input_tokens, 
                                 num_beams=3,
                                 no_repeat_ngram_size=3, 
                                 length_penalty=2.0, 
                                 max_length=500, 
                                 early_stopping=True)
  # decode summarized token
  output = tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return output

In [41]:
def extract_keywords(text, kw_pop=15):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

  kw = ex.get_n_best(n=kw_pop)

  kw_ls = []

  for i in kw:
    kw_ls.append(i[0])

  return kw_ls

In [53]:
def filter_keywords(original, summarized, kw_pop=15):
  orig_ls = extract_keywords(original, kw_pop)
  sum_ls = extract_keywords(summarized, kw_pop)

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [51]:
text = '''

 Cloud computing is named as such because the information being accessed is found remotely in the cloud or a virtual space. Companies that provide cloud services enable users to store files and applications on remote servers and then access all the data via the Internet. This means the user is not required to be in a specific place to gain access to it, allowing the user to work remotely.

Cloud computing takes all the heavy lifting involved in crunching and processing data away from the device you carry around or sit and work at. It also moves all of that work to huge computer clusters far away in cyberspace. The Internet becomes the cloud, and voilà—your data, work, and applications are available from any device with which you can connect to the Internet, anywhere in the world.

Cloud computing can be both public and private. Public cloud services provide their services over the Internet for a fee. Private cloud services, on the other hand, only provide services to a certain number of people. These services are a system of networks that supply hosted services. There is also a hybrid option, which combines elements of both the public and private services. 
'''

summary = summarize(text)
print(summary)

companies that provide cloud services enable users to store files and applications on remote servers and then access all the data via the Internet. cloud computing takes all the heavy lifting involved in crunching and processing data away from the device you carry around or sit and work at. the Internet becomes the cloud, and voilà—your data, work, and applications are available from any device with which you can connect, anywhere in the world.


In [54]:
filtered_kw = filter_keywords(text, summary)

In [42]:
filtered_kw = extract_keywords(text)

In [55]:
filtered_kw

['users',
 'work',
 'companies',
 'cloud computing',
 'internet',
 'data',
 'cloud services',
 'device',
 'voilà',
 'applications',
 'files']

In [5]:
T5_Q_MODEL_PATH = 'ramsrigouthamg/t5_squad_v1'
question_model = T5ForConditionalGeneration.from_pretrained(T5_Q_MODEL_PATH)
question_tokenizer = T5Tokenizer.from_pretrained(T5_Q_MODEL_PATH)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [24]:
def gen_question(context, answer):
  # preprocess
  model_input_text = "context: {} answer: {}".format(context, answer)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  input_tokens = question_tokenizer.encode(model_input_text, return_tensors='pt', max_length = 1000, padding='max_length', truncation=True)

  question_encoded = question_model.generate(input_tokens, 
                                             num_beams=5,
                                             no_repeat_ngram_size=3, 
                                             length_penalty=2.0, 
                                             max_length=100, 
                                             early_stopping=True)
  # decode summarized token
  output = question_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [46]:
filtered_kw

['cloud computing',
 'services',
 'software',
 'cloud',
 'users',
 'internet',
 'data',
 'servers',
 'system',
 'files',
 'types',
 'companies',
 'data storage',
 'device',
 'cloud computing services']

In [49]:
for kw in filtered_kw:
  print(kw)
  print(gen_question(text, kw))

cloud computing
What type of services are available to both public and private customers?
services
What type of services are available to both public and private customers?
software
What type of services are available to both public and private customers?
cloud
What type of services are available to both public and private customers?
users
What type of services are available to both public and private customers?
internet
What type of services are available to both public and private customers?
data
What type of services are available to both public and private customers?
servers
What type of services are available to both public and private customers?
system
What type of services are available to both public and private customers?
files
What type of services are available to both public and private customers?
types
What type of services are available to both public and private customers?
companies
What type of services are available to both public and private customers?
data storage
Wh